# 🚨 Emergency Vehicle Detection - YOLOv5 Training

**Dataset**: Emergency Vehicle Dataset (600 images, 6 classes)  
**Model**: YOLOv5n (nano - fastest inference)  
**Training Time**: ~20-30 minutes on T4 GPU

---

## ⚠️ BEFORE YOU START

**IMPORTANT - Enable GPU First:**
1. Click: **Runtime** → **Change runtime type**
2. Select: **Hardware accelerator** → **T4 GPU**
3. Click: **Save**

**Without GPU, training will take 5-10 hours instead of 30 minutes!**

---

## 📋 Prerequisites
- Google Colab with GPU enabled (see above)
- Dataset file: `dataset.zip` (32 MB - download from Drive link in `README_DATASET.md`)

## 1️⃣ Setup Environment

In [ ]:
# Clone YOLOv5 repository
!git clone https://github.com/ultralytics/yolov5

# Change to yolov5 directory
%cd yolov5

# Install dependencies
!pip install -q -r requirements.txt

## 2️⃣ Upload & Prepare Dataset

Upload `dataset.zip` manually (click files icon → upload)

In [ ]:
# Extract and verify dataset
import zipfile
import os

zip_path = 'dataset.zip'

# Extract
with zipfile.ZipFile(zip_path, 'r') as zip_ref:
    zip_ref.extractall('/content/yolov5/')

## 3️⃣ Train YOLOv5 Model

**Parameters**:
- `--img 640`: Image size
- `--batch 16`: Batch size
- `--epochs 100`: Training epochs
- `--data`: Path to data.yaml
- `--weights yolov5n.pt`: Pretrained weights (nano model)

**Estimated Time**: 20-30 minutes on T4 GPU

In [ ]:
# Train the model
!python train.py \
  --img 640 \
  --batch 16 \
  --epochs 100 \
  --data /content/yolov5/dataset/data.yaml \
  --weights yolov5n.pt

## 4️⃣ Export Model (TorchScript)

Export to TorchScript format for deployment

In [ ]:
# Export best model to TorchScript
!python export.py \
      --weights runs/train/exp/weights/best.pt \
      --include torchscript

## 5️⃣ Download Results

Download training results and model files

In [ ]:
# Zip the results
!zip -r exp_results.zip runs/train/exp

## ✅ Training Complete!

**Next Steps**:
1. Download the results zip file
2. Extract `best.pt` and `best.torchscript` from weights folder
3. Copy to your project: `models/best.torchscript`
4. Test deployment: `python3 src/deployment/main.py`